In [32]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report

In [33]:
file = open("datasets/diabetic_data.csv", "r")
dataset = pd.read_csv(file)
dataset.replace('?', np.nan, inplace=True)
dataset.dropna(axis = 1,inplace=True)

FileNotFoundError: [Errno 2] No such file or directory: 'datasets/diabetic_data.csv'

In [ ]:
datatype = dataset.apply(lambda s: pd.to_numeric(s, errors='coerce').notnull().all())
newdata = []
for data in datatype:
    newdata.append(datatype[1])
datatype = newdata
datatype

[True, True, True, True, True, True, True, True, True]

> Training


In [ ]:
x = dataset.iloc[:, :-1].values
y =dataset.iloc[:,dataset.shape[1]-1].values

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.25, random_state=0)

In [ ]:
params = {'kernel': ['poly', 'rbf'], 'C': [100], 'degree': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'gamma': ['scale', 'auto']}
clf = GridSearchCV(SVC(probability=True, random_state=1), params, cv = 5, refit=True)

In [ ]:
rs = StandardScaler()
xtrain = rs.fit_transform(xtrain)

In [ ]:
clf.fit(xtrain, ytrain)

GridSearchCV(cv=5, estimator=SVC(probability=True, random_state=1),
             param_grid={'C': [100], 'degree': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'gamma': ['scale', 'auto'],
                         'kernel': ['poly', 'rbf']})

> Testing


In [ ]:
xtest = rs.transform(xtest)
ypred = clf.predict(xtest)
print("Predictions\n", ypred)
print("True Values\n" ,np.matrix(ytest))
print(classification_report(ytest, ypred, target_names=['Benign', 'Malignant'], zero_division=1))
print(confusion_matrix(ytest, ypred))
print(clf)

Predictions
 [1 0 0 1 0 0 1 1 0 0 1 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0
 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 1 1 0 1 1 1 1 0 0 0 0 0 0 1
 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 1 0 0 0 0 1 0
 0 1 0 1 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0
 0 0 0 1 0 0 1 0 1 0 1 1 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0
 0 1 0 0 0 0 0]
True Values
 [[1 0 0 1 0 0 1 1 0 0 1 1 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1
  1 0 0 0 0 0 0 1 1 0 0 1 1 1 0 0 1 0 0 0 0 1 1 1 1 0 0 1 1 1 1 0 0 0 0 0
  0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 1 0 0
  0 1 0 1 1 1 1 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 1 1 0 0 0 0 0
  1 0 0 0 0 1 0 1 0 0 1 0 0 0 1 1 1 1 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0
  1 1 0 1 1 0 1 1 1 0 0 0]]
              precision    recall  f1-score   support

      Benign       0.83      0.91      0.86       130
   Malignant       0.76      0.60      0.67        62

    accuracy         

#### Testing on original dataset

In [ ]:
x = dataset.iloc[:, :-1].values
x = rs.transform(x)
y = dataset.iloc[:,dataset.shape[1]-1].values

In [ ]:
ypred = clf.predict(x)
# print("Predictions\n", ypred)
# print("True Values\n" ,np.matrix(y))
print(classification_report(y, ypred, target_names=['Benign', 'Malignant'], zero_division=1))
print(confusion_matrix(y, ypred))

              precision    recall  f1-score   support

      Benign       0.79      0.88      0.84       500
   Malignant       0.73      0.57      0.64       268

    accuracy                           0.77       768
   macro avg       0.76      0.73      0.74       768
weighted avg       0.77      0.77      0.77       768

[[442  58]
 [115 153]]
